In [1]:
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
import pickle
import numpy as np
import json
import re
import nltk
import random

# Lemmatizer object
lemmatizer = WordNetLemmatizer()

In [2]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"wasn't","was not",text)
    text = re.sub(r"\'ll", "will", text)
    text = re.sub(r"\'ve", "have", text)
    text = re.sub(r"\'re", "are", text)
    text = re.sub(r"\'d", "would", text)
    text = re.sub(r"n't", "not", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)

    # Tokenize the text
    words = nltk.word_tokenize(text)
    
    # Lemmatize and remove stop words
    words = [lemmatizer.lemmatize(word.lower()) for word in words]
    #words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words]
    
    return ' '.join(words)

In [3]:
with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

with open("intents.json") as ChatbotData:
  intents = json.load(ChatbotData)

with open('label_encoder.pickle', 'rb') as enc_file:
    lb_encoder = pickle.load(enc_file)

model=keras.models.load_model('model.h5')

maxlen=12

response = {}
for intent in intents['intents']:
      response[intent['tag']] = intent["responses"]


In [5]:
def chat_response(user_inp):
    preprocessed_inp = clean_text(user_inp)
    prediction_input = tokenizer.texts_to_sequences([preprocessed_inp])
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input],truncating='post',padding='post',maxlen=maxlen)

    output = model.predict(prediction_input)
    output = output.argmax()

    response_tag = lb_encoder.inverse_transform([output])[0]
    result = random.choice(response[response_tag])
    return result

# Chatting with bot 

In [6]:
import tkinter as tk
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "👤 : " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chat_response(msg)
        ChatLog.insert(END, "🤖 : " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = tk.Tk()
base.title("Welcome To ML Chatbot")

frame = tk.Frame(base)
frame.pack(pady=10)

header_label = tk.Label(frame, text="Welcome to ML Chatbot", font=("Helvetica", 14, "bold"))
header_label.grid(row=0, column=0, columnspan=3, padx=5, pady=5)

base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width=10, height=2,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()


1/1 [==============================] - 0s 173ms/step
